In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory


llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("files/chapter_three.pdf")

docs = loader.load_and_split(
    text_splitter=splitter,
)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)

vectorstore = FAISS.from_documents(
    docs,
    cached_embeddings,
)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

memory = ConversationBufferMemory(
    return_messages=True,
)


def load_memory(_):
    results = memory.load_memory_variables({})
    return results["history"]


chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": RunnableLambda(load_memory),
    }
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    return result.content

In [2]:
invoke_chain("Is Aaronson guilty?")

'According to the context, Winston believes that Aaronson, along with Jones and Rutherford, is guilty of the crimes they were charged with, despite having seen a photograph that disproved their guilt. He states, "Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with," indicating that he has accepted the Party\'s narrative.'

In [3]:
invoke_chain("What message did Aaronson write on the table?")

"The context does not provide information about any specific message that Aaronson wrote on the table. Therefore, I don't know the answer to that question."

In [4]:
invoke_chain("Who is Julia?")

'Julia is a character who is significant to Winston. She is someone with whom Winston has a romantic relationship. In the context, Winston expresses deep feelings for her, indicating that he loves her and believes she is still alive and needs his help. Their relationship is marked by rebellion against the Party, but it ultimately faces challenges due to the oppressive regime they live under.'

In [5]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Is Aaronson guilty?'),
  AIMessage(content='According to the context, Winston believes that Aaronson, along with Jones and Rutherford, is guilty of the crimes they were charged with, despite having seen a photograph that disproved their guilt. He states, "Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with," indicating that he has accepted the Party\'s narrative.'),
  HumanMessage(content='What message did Aaronson write on the table?'),
  AIMessage(content="The context does not provide information about any specific message that Aaronson wrote on the table. Therefore, I don't know the answer to that question."),
  HumanMessage(content='Who is Julia?'),
  AIMessage(content='Julia is a character who is significant to Winston. She is someone with whom Winston has a romantic relationship. In the context, Winston expresses deep feelings for her, indicating that he loves her and believes she is still alive and needs his help. T

In [6]:
invoke_chain("What questions did I ask?")

'You asked the following questions:\n\n1. Is Aaronson guilty?\n2. What message did Aaronson write on the table?\n3. Who is Julia?'